# Importation des librairies

In [57]:
import numpy as np
import cv2
import os
import pandas as pd
import itertools    
import mahotas as mh
import pickle
import requests,json
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import svm

In [3]:
from sklearn.ensemble import RandomForestClassifier 

# extraction des caractéristiques

In [1]:
def load_features_from_folder(folder):
    #features = np.ones(525)
    subd=[name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
    i=0
    j=0
    train_y=[]
    labels={}
    for sub in subd:
        labels[j]=sub
        for filename in os.listdir(os.path.join(folder,sub)):
            train_y.append(j)
            
            img = cv2.imread(os.path.join(os.path.join(folder,sub,),filename))
            cv2.resize(img,None,fx=0.1,fy=0.1)
            #mean,std,6
            (means, stds) = cv2.meanStdDev(img)
            mean_std=np.array(list(zip(means, stds))).flatten()
            
            #shape
            gray = cv2.cvtColor(np.uint8(img), cv2.COLOR_BGR2GRAY)
            shape=cv2.HuMoments(cv2.moments(gray)).flatten()
            shape = -np.sign(shape) * np.log10(np.abs(shape))
           
            #histogram
            hist = cv2.calcHist([np.uint8(img)], [0, 1, 2], None, [8,8,8], [0, 256, 0, 256, 0, 256]).flatten()
            
            #texture
            texture = mh.features.haralick(img.astype(np.uint8)).ravel()
            feature=np.hstack((mean_std,shape,texture,hist))
            #feature=np.hstack((shape))
            #feature=np.hstack((shape,texture))
            if i==0:
                features = np.zeros(feature.shape[0])
                i=i+1
            features = np.vstack((features,feature))
        j+=1

    return np.delete(features, (0), axis=0),np.asarray(train_y)

# features extraction 

In [ ]:
feat_x,target_y=load_features_from_folder("asl_alphabet_train/asl_alphabet_train")

# serializing the features

In [4]:
import pickle

In [7]:
pcklt=open("target_y.pic","wb")
pickle.dump(target_y,pcklt)
pcklt.close()

In [13]:
pckl=open("features.pic","wb")
pickle.dump(feat_x,pckl)
pckl.close()

# load features

In [6]:
pckl_in=open("features.pic","rb")
feat_x=pickle.load(pckl_in)
feat_x.shape

(87000, 525)

In [7]:
pckl_target_in=open("target_y.pic","rb")
target_y=pickle.load(pckl_target_in)
target_y.shape

(87000,)

#  SVM Classifier

In [58]:
svmc = svm.SVC(gamma='scale', decision_function_shape='ovo')
svmc.fit(feat_x,target_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# RandomForest Classifier

In [45]:
rfc = RandomForestClassifier()
rfc.fit(feat_x,target_y)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# KNN Classifier

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [51]:
knn = KNeighborsClassifier(n_neighbors=7,weights='distance')
knn.fit(feat_x,target_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='distance')

# features extraction of data test

In [9]:
def load_features_from_folder_for_test(folder):

    i=0
    for filename in os.listdir(os.path.join(folder)):
            
        img = cv2.imread(os.path.join(os.path.join(folder),filename))
        cv2.resize(img,None,fx=0.1,fy=0.1)
        #mean,std,6
        (means, stds) = cv2.meanStdDev(img)
        mean_std=np.array(list(zip(means, stds))).flatten()
            
        #shape
        gray = cv2.cvtColor(np.uint8(img), cv2.COLOR_BGR2GRAY)
        shape=cv2.HuMoments(cv2.moments(gray)).flatten()
        shape = -np.sign(shape) * np.log10(np.abs(shape))
        #histogram
        hist = cv2.calcHist([np.uint8(img)], [0, 1, 2], None, [8,8,8], [0, 256, 0, 256, 0, 256]).flatten()
        """#texture
        texture = mh.features.haralick(img.astype(np.uint8)).ravel()"""
        #feature=np.hstack((texture,shape,hist))
        #feature=np.hstack((texture))
        feature=np.hstack((mean_std,shape,hist))
        if i==0:
            features = np.zeros(feature.shape[0])
            i=i+1
        features = np.vstack((features,feature))
    return np.delete(features, (0), axis=0)

In [10]:
test_x=load_features_from_folder_for_test("asl_alphabet_test/asl_alphabet_test")

# test RFC

In [46]:
test_rfc = rfc.predict(test_x)
print('accuracy RFC: ',test_rfc)

accuracy RFC:  ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'nothing' 'N' 'O' 'P'
 'Q' 'R' 'space' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


# Test KNN 

In [52]:
knn_pred=knn.predict(test_x)

In [53]:
print("prédictions de knn: \n{}".format(knn_pred))

prédictions de knn: 
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'nothing' 'N' 'O' 'P'
 'Q' 'R' 'space' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [55]:
accuracy=accuracy_score(range(1,29),knn_pred)

In [56]:
print('accuracy: {}'.format(accuracy*100))

accuracy: 85.71428571428571


# test SVM

In [60]:
test_svm=svmc.predict(test_x)
print('predictions of SVM Classifier: {}'.format(test_svm))

predictions of SVM Classifier: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'nothing' 'N' 'O' 'P'
 'Q' 'R' 'space' 'S' 'T' 'U' 'V' 'W' 'X' 'X' 'Z']
